In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,models,layers

In [2]:
embedding_layer = layers.Embedding(1000,32)

In [3]:
vocab_size = 10000
(trainX,trainy),(testX,testy) = datasets.imdb.load_data(num_words = vocab_size)

17465344/17464789 [==============================] - 0s 0us/step


In [4]:
print(trainX[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
print(len(trainX[0]))

218


In [22]:
word_index = datasets.imdb.get_word_index()
word_index = {k:v+3 for k,v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3
rev_word_index = dict([v,k] for (k,v) in word_index.items())
def decode(sent_ints):
    return ' '.join([rev_word_index[i] for i in sent_ints]) 

In [25]:
maxlen=500
trainX = keras.preprocessing.sequence.pad_sequences(trainX,
                    value=word_index['<PAD>'],
                    padding='post',
                    maxlen=maxlen)
testX = keras.preprocessing.sequence.pad_sequences(testX,
                    value=word_index['<PAD>'],
                    padding='post',
                    maxlen=maxlen)        

In [28]:
embedding_dim = 16
def build_model():
    model = models.Sequential([
        layers.Embedding(vocab_size,embedding_dim,input_length=maxlen),
        layers.GlobalAveragePooling1D(),
        layers.Dense(16,activation='relu'),
        layers.Dense(1,activation='sigmoid')
    ])

    model.summary()
    model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model


In [29]:
model = build_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [30]:
history = model.fit(trainX,trainy,validation_data=[testX,testy],
                   epochs=20,batch_size=512)

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 3s 103us/sample - loss: 0.6919 - accuracy: 0.5599 - val_loss: 0.6900 - val_accuracy: 0.6062
Epoch 2/20
25000/25000 [==============================] - 2s 90us/sample - loss: 0.6848 - accuracy: 0.6646 - val_loss: 0.6782 - val_accuracy: 0.7172
Epoch 3/20
25000/25000 [==============================] - 2s 90us/sample - loss: 0.6640 - accuracy: 0.7534 - val_loss: 0.6499 - val_accuracy: 0.7585
Epoch 4/20
25000/25000 [==============================] - 2s 90us/sample - loss: 0.6236 - accuracy: 0.7840 - val_loss: 0.6047 - val_accuracy: 0.7745
Epoch 5/20
25000/25000 [==============================] - 2s 90us/sample - loss: 0.5647 - accuracy: 0.8104 - val_loss: 0.5446 - val_accuracy: 0.8110
Epoch 6/20
25000/25000 [==============================] - 2s 91us/sample - loss: 0.4994 - accuracy: 0.8384 - val_loss: 0.4876 - val_accuracy: 0.8308
Epoch 7/20
25000/25000 [==============================]

In [37]:
# Lets get the word embeddings
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [39]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(vocab_size):
  word = rev_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()